# Tutorial from Planetary Computer (PC)

**Parts of the code in this notebook were adopted from Microsofts Planetary Computer website directly.**

[Sentinel-2 Level-2A](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook)

## Environment setup

In [1]:
import re
import folium
import rasterio
import numpy as np
import pandas as pd
import pystac_client
from PIL import Image
import geopandas as gpd
import planetary_computer
# from pyproj import Transformer
import base64, io, os, requests
import matplotlib.pyplot as plt
from rasterio import warp, features
from collections import defaultdict
from folium.elements import Element
from shapely.geometry import Point, mapping
from rasterio import windows, features, warp
from shapely.ops import transform as shp_transform
from pystac.extensions.eo import EOExtension as eo

plt.style.use("~/geoscience/albedo_downscaling/MNRAS.mplstyle")
%matplotlib inline

## Data access

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## Choose a region and time of interest

In [3]:
shapefile_path = "/bsuhome/tnde/scratch/felix/modis/East_River_SHP/ER_bbox.shp"
# AOI from shapefile (WGS84 for STAC)
Boundary = gpd.read_file(shapefile_path)
Boundary_wgs84 = Boundary.to_crs(epsg=4326)

# bounds tuple (minlon, minlat, maxlon, maxlat)
AOI_BOUNDS_LL = tuple(Boundary_wgs84.total_bounds)
AOI_GEOM = mapping(Boundary_wgs84.union_all())
area_of_interest = AOI_GEOM

In [4]:
time_of_interest = "2021-09-01/2023-06-15"

In [5]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 100}}, # No limit on cloud fraction 10 
)

# ---- granule IDs ----
items = list(search.items()) # Check how many granules were returned
print(f"Returned {len(items)} Items")

Returned 185 Items


# Export images to html

There are different options but one can just choose only one option depending on their preferences.

## Sentinel-2 gallery with cloud cover fraction overlayed

In [6]:
# ---- Helper functions ----
def _stretch_01(a, p_low=2, p_high=98):
    """
    Percentile-based contrast stretching to the range [0, 1].

    This function rescales an input numeric array by linearly mapping the values
    between the `p_low` and `p_high` percentiles to [0, 1]. Values below the
    low percentile map to 0, values above the high percentile map to 1, and
    intermediate values are scaled linearly. NaNs and infs are ignored when
    computing percentiles.

    The operation is commonly used to create display-ready images from
    reflectance/ radiance bands or other intensity-like arrays.

    Parameters
    ----------
    a : array_like
        Input numeric array of any shape. Will be converted to ``float32`` for
        processing. NaNs/Infs are allowed; they are ignored in percentile
        computation but remain in the output after clipping.
    p_low : float, optional
        Lower percentile (0–100) used as the black point. Default is 2.
    p_high : float, optional
        Upper percentile (0–100) used as the white point. Default is 98.

    Returns
    -------
    ndarray
        Array of the same shape as `a`, dtype ``float32``, with values in
        ``[0, 1]`` (NaNs remain NaN). If the input contains no finite values,
        returns an array of zeros with the same shape and dtype.

    Notes
    -----
    - If the computed `hi` percentile is less than or equal to the `lo`
      percentile (e.g., near-constant arrays), a small epsilon is added to
      `hi` to avoid division by zero, effectively returning all zeros.
    - Only finite values contribute to percentile estimation; non-finite values
      are left unchanged after the final clipping.
    - This function is symmetric with respect to array shape and can be applied
      to single bands or multi-band stacks (apply per-band if needed).
    """
    a = a.astype("float32")
    m = np.isfinite(a)
    if not m.any():
        return np.zeros_like(a, dtype="float32")
    lo, hi = np.percentile(a[m], [p_low, p_high])
    if hi <= lo: hi = lo + 1e-6
    return np.clip((a - lo) / (hi - lo), 0, 1)

def _get_cloud_pct(item):
    """Return cloud percentage as float or None from common STAC keys."""
    props = getattr(item, "properties", {}) or {}
    for k in ("eo:cloud_cover", "s2:cloud_percentage", "cloud_cover"):
        if k in props:
            try:
                return float(props[k])
            except Exception:
                pass
    return None

def read_rgb_from_visual_or_bands(item, thumb_px=512):
    """
    Build a small RGB preview image for a Sentinel-2 STAC item.

    Tries assets in this order:
      1) 'rendered_preview' or 'preview' (PNG, fastest; HTTP GET),
      2) 'visual' (TCI GeoTIFF; reads first 3 bands),
      3) raw bands 'B04','B03','B02' (forms RGB, percentile-stretched).
    Falls back to a gray placeholder if all attempts fail. The output image is
    resized in-place to have its longest side ≤ `thumb_px`.

    Parameters
    ----------
    item : pystac.Item
        Sentinel-2 L2A STAC item with assets (must include at least one of the
        tried assets listed above).
    thumb_px : int, default 512
        Maximum thumbnail size (pixels) for the longer edge.

    Returns
    -------
    PIL.Image.Image
        RGB uint8 thumbnail suitable for embedding in HTML or notebooks.
    """
    # rendered_preview (PNG)
    for key in ("rendered_preview", "preview"):
        if key in item.assets:
            try:
                href = item.assets[key].href
                r = requests.get(href, timeout=60)
                r.raise_for_status()
                img = Image.open(io.BytesIO(r.content)).convert("RGB")
                img.thumbnail((thumb_px, thumb_px))
                return img
            except Exception:
                pass

    # visual (TCI GeoTIFF)
    if "visual" in item.assets:
        try:
            href = item.assets["visual"].href
            with rasterio.open(href) as src:
                arr = src.read()  # bands, H, W
            if arr.shape[0] >= 3:
                rgb = np.moveaxis(arr[:3], 0, -1)  # H,W,3
                if rgb.dtype != np.uint8:
                    rgb = (_stretch_01(rgb) * 255).astype("uint8")
                img = Image.fromarray(rgb)
                img.thumbnail((thumb_px, thumb_px))
                return img
        except Exception:
            pass

    # B04/B03/B02
    have = all(k in item.assets for k in ("B04","B03","B02"))
    if have:
        try:
            with rasterio.open(item.assets["B04"].href) as r4, \
                 rasterio.open(item.assets["B03"].href) as r3, \
                 rasterio.open(item.assets["B02"].href) as r2:
                R = r4.read(1); G = r3.read(1); B = r2.read(1)
            rgb = np.dstack([R, G, B])
            rgb = (_stretch_01(rgb) * 255).astype("uint8")
            img = Image.fromarray(rgb)
            img.thumbnail((thumb_px, thumb_px))
            return img
        except Exception:
            pass

    # If everything fails, return a small gray tile noting failure
    img = Image.new("RGB", (thumb_px, thumb_px), (200, 200, 200))
    return img

def to_data_url(pil_img, fmt="PNG"):
    """
    Convert a PIL image into a base64-encoded data URL string.

    This is useful for embedding images directly into HTML without writing them
    to disk. The image is first saved to an in-memory buffer, then base64-encoded
    and returned as a data URL (e.g., "data:image/png;base64,...").

    Parameters
    ----------
    pil_img : PIL.Image.Image
        Input image to encode.
    fmt : str, default "PNG"
        Output image format (e.g., "PNG", "JPEG").

    Returns
    -------
    str
        Base64-encoded data URL string representing the image.
    """
    buf = io.BytesIO()
    pil_img.save(buf, format=fmt)
    b64 = base64.b64encode(buf.getvalue()).decode("ascii")
    return f"data:image/{fmt.lower()};base64,{b64}"

# ---- Build the HTML gallery ----
tiles = []
for it in items:
    try:
        img = read_rgb_from_visual_or_bands(it, thumb_px=512)
        data_url = to_data_url(img, fmt="PNG")
        title = getattr(it, "id", "unknown")
        dt = getattr(it, "datetime", None)
        subtitle = dt.isoformat() if dt else ""
        cloud = _get_cloud_pct(it)
        cloud_str = (f"{cloud:.1f}%") if (cloud is not None) else "N/A"
        tiles.append((title, subtitle, data_url, cloud_str))   # <<-- ADDED cloud_str
    except Exception as e:
        tiles.append((getattr(it, "id", "unknown"), "FAILED", "", "N/A"))
        
html = """
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Sentinel-2 RGB Gallery</title>
<style>
body{font-family:Arial, sans-serif; margin:20px; background:#111; color:#eee;}
.grid{display:grid; grid-template-columns:repeat(auto-fill, minmax(260px,1fr)); gap:16px;}
.card{background:#1b1b1b; border-radius:12px; overflow:hidden; box-shadow:0 2px 10px rgba(0,0,0,.4); position:relative;} /* << ADDED position */
.card img{width:100%; display:block;}
.meta{padding:10px 12px;}
.meta h3{margin:0 0 6px; font-size:14px; color:#fff;}
.meta p{margin:0; font-size:12px; color:#ccc; word-break:break-all;}
.cloud{position:absolute; top:8px; left:8px; background:rgba(0,0,0,0.35); color:#ff0000; padding:2px 6px; border-radius:6px; font-size:15px; font-weight:700;} /* << NEW */
</style>
</head>
<body>
<h1>Sentinel-2 L2A RGB Gallery</h1>
<div class="grid">
"""

for title, subtitle, data_url, cloud_str in tiles:   # <<-- unpack cloud_str
    img_tag = f'<img src="{data_url}" alt="{title}">' if data_url else '<div style="height:200px;background:#333;"></div>'
    html += f"""
    <div class="card">
        <div class="cloud">Cloud: {cloud_str}</div>  <!-- <<-- ADDED overlay label -->
        {img_tag}
        <div class="meta">
            <h3>{title}</h3>
            <p>{subtitle}</p>
        </div>
    </div>
    """
html += """
</div>
</body>
</html>
"""

out_html = "/bsuhome/tnde/geoscience/albedo_downscaling/sentinel_2/webmap_html/s2_gallery_with_cloud_cover_fraction.html"
with open(out_html, "w", encoding="utf-8") as f:
    f.write(html)

print(f"Wrote {out_html} with {len(tiles)} tiles")

Wrote /bsuhome/tnde/geoscience/albedo_downscaling/sentinel_2/webmap_html/s2_gallery_with_cloud_cover_fraction.html with 185 tiles


## Sentinel-2 gallery with date dropdown and cloud cover fraction overlayed

In [7]:
# ---- granule IDs ----
items = list(search.items())
items = [planetary_computer.sign(it) for it in items]

# ---- Helper functions ----
def _stretch_01(a, p_low=2, p_high=98):
    """
    Percentile-based contrast stretching to the range [0, 1].

    This function rescales an input numeric array by linearly mapping the values
    between the `p_low` and `p_high` percentiles to [0, 1]. Values below the
    low percentile map to 0, values above the high percentile map to 1, and
    intermediate values are scaled linearly. NaNs and infs are ignored when
    computing percentiles.

    The operation is commonly used to create display-ready images from
    reflectance/ radiance bands or other intensity-like arrays.

    Parameters
    ----------
    a : array_like
        Input numeric array of any shape. Will be converted to ``float32`` for
        processing. NaNs/Infs are allowed; they are ignored in percentile
        computation but remain in the output after clipping.
    p_low : float, optional
        Lower percentile (0–100) used as the black point. Default is 2.
    p_high : float, optional
        Upper percentile (0–100) used as the white point. Default is 98.

    Returns
    -------
    ndarray
        Array of the same shape as `a`, dtype ``float32``, with values in
        ``[0, 1]`` (NaNs remain NaN). If the input contains no finite values,
        returns an array of zeros with the same shape and dtype.

    Notes
    -----
    - If the computed `hi` percentile is less than or equal to the `lo`
      percentile (e.g., near-constant arrays), a small epsilon is added to
      `hi` to avoid division by zero, effectively returning all zeros.
    - Only finite values contribute to percentile estimation; non-finite values
      are left unchanged after the final clipping.
    - This function is symmetric with respect to array shape and can be applied
      to single bands or multi-band stacks (apply per-band if needed).
    """
    a = a.astype("float32")
    m = np.isfinite(a)
    if not m.any():
        return np.zeros_like(a, dtype="float32")
    lo, hi = np.percentile(a[m], [p_low, p_high])
    if hi <= lo: hi = lo + 1e-6
    return np.clip((a - lo) / (hi - lo), 0, 1)

def _get_cloud_pct(item):
    """Return cloud percentage as float or None from common STAC keys."""
    props = getattr(item, "properties", {}) or {}
    for k in ("eo:cloud_cover", "s2:cloud_percentage", "cloud_cover"):
        if k in props:
            try:
                return float(props[k])
            except Exception:
                pass
    return None

def read_rgb_from_visual_or_bands(item, thumb_px=512):
    """
    Build a small RGB preview image for a Sentinel-2 STAC item.

    Tries assets in this order:
      1) 'rendered_preview' or 'preview' (PNG, fastest; HTTP GET),
      2) 'visual' (TCI GeoTIFF; reads first 3 bands),
      3) raw bands 'B04','B03','B02' (forms RGB, percentile-stretched).
    Falls back to a gray placeholder if all attempts fail. The output image is
    resized in-place to have its longest side ≤ `thumb_px`.

    Parameters
    ----------
    item : pystac.Item
        Sentinel-2 L2A STAC item with assets (must include at least one of the
        tried assets listed above).
    thumb_px : int, default 512
        Maximum thumbnail size (pixels) for the longer edge.

    Returns
    -------
    PIL.Image.Image
        RGB uint8 thumbnail suitable for embedding in HTML or notebooks.
    """
    # rendered_preview (PNG)
    for key in ("rendered_preview", "preview"):
        if key in item.assets:
            try:
                href = item.assets[key].href
                r = requests.get(href, timeout=60)
                r.raise_for_status()
                img = Image.open(io.BytesIO(r.content)).convert("RGB")
                img.thumbnail((thumb_px, thumb_px))
                return img
            except Exception:
                pass

    # visual (TCI GeoTIFF)
    if "visual" in item.assets:
        try:
            href = item.assets["visual"].href
            with rasterio.open(href) as src:
                arr = src.read()  # bands, H, W
            if arr.shape[0] >= 3:
                rgb = np.moveaxis(arr[:3], 0, -1)  # H,W,3
                if rgb.dtype != np.uint8:
                    rgb = (_stretch_01(rgb) * 255).astype("uint8")
                img = Image.fromarray(rgb)
                img.thumbnail((thumb_px, thumb_px))
                return img
        except Exception:
            pass

    # B04/B03/B02
    have = all(k in item.assets for k in ("B04","B03","B02"))
    if have:
        try:
            with rasterio.open(item.assets["B04"].href) as r4, \
                 rasterio.open(item.assets["B03"].href) as r3, \
                 rasterio.open(item.assets["B02"].href) as r2:
                R = r4.read(1); G = r3.read(1); B = r2.read(1)
            rgb = np.dstack([R, G, B])
            rgb = (_stretch_01(rgb) * 255).astype("uint8")
            img = Image.fromarray(rgb)
            img.thumbnail((thumb_px, thumb_px))
            return img
        except Exception:
            pass

    # If everything fails, return a small gray tile noting failure
    img = Image.new("RGB", (thumb_px, thumb_px), (200, 200, 200))
    return img

def to_data_url(pil_img, fmt="PNG"):
    """
    Convert a PIL image into a base64-encoded data URL string.

    This is useful for embedding images directly into HTML without writing them
    to disk. The image is first saved to an in-memory buffer, then base64-encoded
    and returned as a data URL (e.g., "data:image/png;base64,...").

    Parameters
    ----------
    pil_img : PIL.Image.Image
        Input image to encode.
    fmt : str, default "PNG"
        Output image format (e.g., "PNG", "JPEG").

    Returns
    -------
    str
        Base64-encoded data URL string representing the image.
    """
    buf = io.BytesIO()
    pil_img.save(buf, format=fmt)
    b64 = base64.b64encode(buf.getvalue()).decode("ascii")
    return f"data:image/{fmt.lower()};base64,{b64}"

# ---- Build a Folium map with a DATE DROPDOWN + terrain basemap (zoomable) ----
# Compute overall bounds to fit map view
def _bbox_to_bounds(bbox):
    """
    Convert a bounding box from [minx, miny, maxx, maxy] format to
    Folium/Leaflet bounds format [[south, west], [north, east]].

    Parameters
    ----------
    bbox : list or tuple of float
        Bounding box coordinates in the order [minx, miny, maxx, maxy].

    Returns
    -------
    list
        Nested list of [[south, west], [north, east]] suitable for Folium bounds.
    """
    return [[bbox[1], bbox[0]], [bbox[3], bbox[2]]]

# Group items by UTC date (YYYY-MM-DD)
by_date = defaultdict(list)
for it in items:
    dt = getattr(it, "datetime", None)
    date_str = dt.date().isoformat() if dt else "unknown"
    by_date[date_str].append(it)

if not items:
    raise SystemExit("No items to display on a map.")

# Initialize map centered on first item's bbox, use TERRAIN basemap
first_bbox = items[0].bbox or getattr(items[0], "bbox", None)
if not first_bbox:
    m = folium.Map(location=[0, 0], zoom_start=4, tiles=None)
    # folium.TileLayer(
    #     tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    #     name="Terrain (OpenTopoMap)",
    #     attr="Map data © OpenStreetMap contributors, SRTM | Map style © OpenTopoMap (CC-BY-SA)",
    #     max_zoom=17,
    #     control=False
    # ).add_to(m)
    
    # # Esri topo style
    # folium.TileLayer(
    #     tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    #     name="Esri WorldTopoMap",
    #     attr="Tiles © Esri — Esri, DeLorme, FAO, NOAA, USGS, © OpenStreetMap contributors, and the GIS User Community",
    #     max_zoom=19,
    #     control=False
    # ).add_to(m)
    
    # Esri world imagery
    folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        name="Esri World Imagery",
        attr="Tiles © Esri — Source: Esri, Maxar, Earthstar Geographics, and the GIS User Community",
        max_zoom=19,
        control=False
    ).add_to(m)
else:
    (minx, miny, maxx, maxy) = first_bbox
    ctr_lat = (miny + maxy) / 2
    ctr_lon = (minx + maxx) / 2
    m = folium.Map(location=[ctr_lat, ctr_lon], zoom_start=9, tiles=None)
    # folium.TileLayer(
    #     tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    #     name="Terrain (OpenTopoMap)",
    #     attr="Map data © OpenStreetMap contributors, SRTM | Map style © OpenTopoMap (CC-BY-SA)",
    #     max_zoom=17,
    #     control=False
    # ).add_to(m)
    
    # # Esri topo style
    # folium.TileLayer(
    #     tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    #     name="Esri WorldTopoMap",
    #     attr="Tiles © Esri — Esri, DeLorme, FAO, NOAA, USGS, © OpenStreetMap contributors, and the GIS User Community",
    #     max_zoom=19,
    #     control=False
    # ).add_to(m)
    
    # Esri world imagery
    folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        name="Esri World Imagery",
        attr="Tiles © Esri — Source: Esri, Maxar, Earthstar Geographics, and the GIS User Community",
        max_zoom=19,
        control=False
    ).add_to(m)

# Build a FeatureGroup per date; only the FIRST date is visible by default
dates_sorted = sorted(by_date.keys())
all_bounds = []
date_groups = {}  # keep for JS wiring if needed

for i, d in enumerate(dates_sorted, 1):
    fg = folium.FeatureGroup(name=d, show=(i == 1))
    date_groups[d] = fg

    for it in by_date[d]:
        bbox = it.bbox
        if bbox:
            all_bounds.append(_bbox_to_bounds(bbox))

        # Prefer Planetary Computer tilejson (zoomable)
        if "tilejson" not in it.assets:
            continue
        try:
            tj_href = it.assets["tilejson"].href
            tj = requests.get(tj_href, timeout=60).json()
            tiles_url = tj["tiles"][0]
        except Exception:
            continue

        folium.TileLayer(
            tiles=tiles_url,
            name=getattr(it, "id", "granule"),
            attr="Microsoft Planetary Computer / Sentinel-2",
            overlay=True,
            control=False,
            show=True,
            max_zoom=14,
            min_zoom=2
        ).add_to(fg)

        # outline footprint
        if bbox:
            # Draw the footprint rectangle
            folium.Rectangle(
                bounds=_bbox_to_bounds(bbox),
                color="#ff7800",
                weight=1,
                fill=False,
                tooltip=f"{getattr(it,'id','')}"
            ).add_to(fg)

            # --- NEW: add a red cloud-cover label at top-left (NW) corner ---
            minx, miny, maxx, maxy = bbox
            # nw_lat, nw_lon = maxy, minx  # (lat, lon) for NW corner
            # (lat, lon) for NW corner
            nw_lat = maxy - 0.02 * (maxy - miny)
            nw_lon = minx + 0.02 * (maxx - minx)
            cloud = _get_cloud_pct(it)
            cloud_str = (f"{cloud:.1f}%") if (cloud is not None) else "N/A"

            folium.Marker(
                location=[nw_lat, nw_lon],
                icon=folium.DivIcon(html=f'''
                    <div style="
                        background: rgba(255,255,255,0.45); 
                        border-radius: 6px; 
                        padding: 2px 6px; 
                        border: 1px solid #c00; 
                        color: #ff0000; 
                        font-weight: 700; 
                        font-size: 15px; 
                        white-space: nowrap; 
                        display: inline-block;">
                        Cloud: {cloud_str}
                    </div>
                ''')
            ).add_to(fg)

    fg.add_to(m)

# Layer control (shows date groups)
folium.LayerControl(collapsed=False).add_to(m)

# Fit to all bounds if available
if all_bounds:
    min_lat = min(b[0][0] for b in all_bounds)
    min_lon = min(b[0][1] for b in all_bounds)
    max_lat = max(b[1][0] for b in all_bounds)
    max_lon = max(b[1][1] for b in all_bounds)
    m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# ---------- Add a fixed-position DATE DROPDOWN that filters FeatureGroups ----------
# First date selected by default
options_html = "\n".join(
    [f'<option value="{d}"{" selected" if i==0 else ""}>{d}</option>' for i, d in enumerate(dates_sorted)]
)

control_html = f"""
<div id="date-dropdown" style="
  position: absolute; top: 10px; left: 46px; z-index: 9999;
  background: rgba(255,255,255,0.95); padding: 6px 8px; border-radius: 6px;
  box-shadow: 0 1px 4px rgba(0,0,0,0.3); font-family: Arial; font-size: 13px;">
  <label for="dateSelect" style="margin-right:6px;">Date</label>
  <select id="dateSelect">{options_html}</select>
</div>
<script>
(function() {{
  const sel = document.getElementById('dateSelect');

  // Find overlay checkboxes in Leaflet layer control whose label text equals the date
  function getOverlayRows() {{
    return Array.from(document.querySelectorAll('.leaflet-control-layers-overlays label'));
  }}

  function setDate(d) {{
    const rows = getOverlayRows();
    // Uncheck all
    rows.forEach(row => {{
      const cb = row.querySelector('input[type="checkbox"]');
      if (cb && cb.checked) cb.click();
    }});
    // Check only the row whose label text equals the selected date
    rows.forEach(row => {{
      const txt = row.textContent.trim();
      const cb = row.querySelector('input[type="checkbox"]');
      if (txt === d && cb && !cb.checked) cb.click();
    }});
  }}

  sel.addEventListener('change', () => setDate(sel.value));

  // Apply default (first option) on load
  setTimeout(() => setDate(sel.value), 200);
}})();
</script>
"""

m.get_root().html.add_child(Element(control_html))
out_html = "/bsuhome/tnde/geoscience/albedo_downscaling/sentinel_2/webmap_html/s2_gallery_with_date_dropdown_and_cloud_cover_fraction.html"
m.save(out_html)
print(f"Wrote interactive Folium map with date dropdown + terrain basemap: {out_html} with {len(tiles)} tiles")

Wrote interactive Folium map with date dropdown + terrain basemap: /bsuhome/tnde/geoscience/albedo_downscaling/sentinel_2/webmap_html/s2_gallery_with_date_dropdown_and_cloud_cover_fraction.html


# The End